<a href="https://colab.research.google.com/github/abdinegara783/ML-Mentoring-Platform-Dicoding/blob/main/Feedback_fix_Abdiipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mengimport Library
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Memuat dataset IMDB Reviews
(train_dataset, test_dataset), info = tfds.load('imdb_reviews', split=['train', 'test'], shuffle_files=True, with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNV9U87/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNV9U87/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNV9U87/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
# Menampilkan beberapa contoh data dari dataset
num_examples = 5

print("Contoh data dari train_dataset:")
for i, (text, label) in enumerate(train_dataset.take(num_examples), 1):
    print(f"Data {i}:")
    print(f"Text: {text}")
    print(f"Label: {label}")
    print()

print("Contoh data dari test_dataset:")
for i, (text, label) in enumerate(test_dataset.take(num_examples), 1):
    print(f"Data {i}:")
    print(f"Text: {text}")
    print(f"Label: {label}")
    print()

Contoh data dari train_dataset:
Data 1:
Text: b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Label: 0

Data 2:
Text: b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasio

In [11]:
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set = set()

for text_tensor, _ in train_dataset:
    text = text_tensor.numpy().decode('utf-8')
    some_tokens = tokenizer.tokenize(text)
    vocabulary_set.update(some_tokens)


vocab_size = len(vocabulary_set)
embedding_dim = 100
max_length = 200
padding_type = 'post'

train_size = info.splits['train'].num_examples
test_size = info.splits['test'].num_examples

# Preprocessing dan membangun pipeline
def preprocess(text, label):
    text = tf.strings.lower(text)
    text = tokenizer.tokenize(text.numpy().decode('utf-8'))
    text = [word for word in text if word not in stopwords]
    text = [word for word in text if len(word) > 2]
    text = [word for word in text if word.isalpha()]
    text = [word for word in text if word in vocabulary_set]
    text = tokenizer.detokenize(text)
    return text, label


stopwords = ['a', 'an', 'the', 'is', 'was', 'were', 'and', 'or', 'in', 'at', 'to', 'on', 'but', 'with', 'that', 'this']

def encode(text, label):
    encoded_text = tokenizer.tokenize(text.numpy())
    encoded_text = tokenizer.tokens_to_ids(encoded_text)
    return encoded_text, label

def filter_max_length(x, y, max_length=max_length):
    return tf.logical_and(tf.size(x) <= max_length, tf.size(y) <= max_length)

train_dataset = train_dataset.map(preprocess).map(encode).filter(filter_max_length).padded_batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).map(encode).filter(filter_max_length).padded_batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Membangun model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

# Evaluasi model
loss, accuracy = model.evaluate(test_dataset)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

AttributeError: ignored

In [14]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Memuat dataset IMDB Reviews
(train_dataset, test_dataset), info = tfds.load('imdb_reviews', split=['train', 'test'], shuffle_files=True, with_info=True, as_supervised=True)

# Pra-pemrosesan data
tokenizer = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()

for text_tensor, _ in train_dataset:
    some_tokens = tokenizer.tokenize(text_tensor.numpy().decode())
    vocabulary_set.update(some_tokens)



In [19]:
def preprocess(text, label):
    text = tf.strings.lower(text)
    text = tf.py_function(tokenizer.tokenize, inp=[text], Tout=tf.string)
    text = tf.py_function(remove_stopwords, inp=[text], Tout=tf.string)
    text = tf.py_function(remove_short_words, inp=[text], Tout=tf.string)
    text = tf.py_function(keep_alpha_words, inp=[text], Tout=tf.string)
    text = tf.py_function(keep_vocabulary_words, inp=[text], Tout=tf.string)
    return text, label

def encode(text, label):
    encoded_text = tf.py_function(tokenizer.tokenize, inp=[text], Tout=tf.string)
    encoded_text = tf.py_function(tokenizer.tokens_to_ids, inp=[encoded_text], Tout=tf.int32)
    return encoded_text, label

def remove_stopwords(text):
    # Fungsi untuk menghapus stopwords
    stopwords = tf.constant(['a', 'an', 'the', 'is', 'was', 'were', 'and', 'or', 'in', 'at', 'to', 'on', 'but', 'with', 'that', 'this'])
    words = tf.strings.split(text, ' ')
    non_stopwords = tf.boolean_mask(words, tf.logical_not(tf.reduce_any(tf.equal(words, stopwords), axis=-1)))
    return tf.reduce_join(non_stopwords, separator=' ')

def remove_short_words(text):
    # Fungsi untuk menghapus kata-kata pendek
    words = tf.strings.split(text, ' ')
    non_short_words = tf.boolean_mask(words, tf.greater(tf.strings.length(words), 2))
    return tf.reduce_join(non_short_words, separator=' ')

def keep_alpha_words(text):
    # Fungsi untuk mempertahankan hanya kata-kata yang terdiri dari alfabet
    words = tf.strings.split(text, ' ')
    alpha_words = tf.boolean_mask(words, tf.strings.regex_full_match(words, r'[a-zA-Z]+'))
    return tf.reduce_join(alpha_words, separator=' ')

def keep_vocabulary_words(text):
    # Fungsi untuk mempertahankan hanya kata-kata yang ada dalam vocabulary_set
    words = tf.strings.split(text, ' ')
    vocabulary_words = tf.boolean_mask(words, tf.reduce_any(tf.equal(words, list(vocabulary_set)), axis=-1))
    return tf.reduce_join(vocabulary_words, separator=' ')

In [20]:
vocab_size = len(vocabulary_set)
embedding_dim = 100
max_length = 200
padding_type = 'post'

train_size = info.splits['train'].num_examples
test_size = info.splits['test'].num_examples

# Preprocessing dan membangun pipeline
def preprocess(text, label):
    text = tf.strings.lower(text)
    text = tokenizer.tokenize(text.numpy())
    text = [word for word in text if word not in stopwords]
    text = [word for word in text if len(word) > 2]
    text = [word for word in text if word.isalpha()]
    text = [word for word in text if word in vocabulary_set]
    text = tokenizer.detokenize(text)
    return text, label


In [17]:
stopwords = ['a', 'an', 'the', 'is', 'was', 'were', 'and', 'or', 'in', 'at', 'to', 'on', 'but', 'with', 'that', 'this']

def encode(text, label):
    encoded_text = tokenizer.tokenize(text.numpy())
    encoded_text = tokenizer.tokens_to_ids(encoded_text)
    return encoded_text, label

def filter_max_length(x, y, max_length=max_length):
    return tf.logical_and(tf.size(x) <= max_length, tf.size(y) <= max_length)


In [21]:
train_dataset = train_dataset.map(preprocess).map(encode).filter(filter_max_length).padded_batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).map(encode).filter(filter_max_length).padded_batch(32).prefetch(tf.data.experimental.AUTOTUNE)

AttributeError: ignored

In [ ]:
# Membangun model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)

# Evaluasi model
loss, accuracy = model.evaluate(test_dataset)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Memuat dataset IMDB Reviews
(train_dataset, test_dataset), info = tfds.load('imdb_reviews', split=['train', 'test'], shuffle_files=True, with_info=True, as_supervised=True)

# Pra-pemrosesan data untuk klasifikasi sentimen
tokenizer_sentiment = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()

for text_tensor, _ in train_dataset:
    some_tokens = tokenizer_sentiment.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
embedding_dim = 100
max_length = 200
padding_type = 'post'

train_size = info.splits['train'].num_examples
test_size = info.splits['test'].num_examples

# Preprocessing dan membangun pipeline untuk klasifikasi sentimen
def preprocess_sentiment(text, label):
    text = tf.strings.lower(text)
    text = tokenizer_sentiment.tokenize(text.numpy().decode('utf-8'))
    text = [word for word in text if word not in stopwords]
    text = [word for word in text if len(word) > 2]
    text = [word for word in text if word.isalpha()]
    text = [word for word in text if word in vocabulary_set]
    text = tokenizer_sentiment.detokenize(text)
    return text, label

stopwords = ['a', 'an', 'the', 'is', 'was', 'were', 'and', 'or', 'in', 'at', 'to', 'on', 'but', 'with', 'that', 'this']

def encode_sentiment(text, label):
    encoded_text = tokenizer_sentiment.tokenize(text.numpy())
    encoded_text = tokenizer_sentiment.tokens_to_ids(encoded_text)
    return encoded_text, label

def filter_max_length_sentiment(x, y, max_length=max_length):
    return tf.logical_and(tf.size(x) <= max_length, tf.size(y) <= max_length)

train_dataset_sentiment = train_dataset.map(preprocess_sentiment).map(encode_sentiment).filter(filter_max_length_sentiment).padded_batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset_sentiment = test_dataset.map(preprocess_sentiment).map(encode_sentiment).filter(filter_max_length_sentiment).padded_batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Membangun model klasifikasi sentimen
model_sentiment = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_sentiment.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model klasifikasi sentimen
num_epochs_sentiment = 10
history_sentiment = model_sentiment.fit(train_dataset_sentiment, epochs=num_epochs_sentiment, validation_data=test_dataset_sentiment)

# Evaluasi model klasifikasi sentimen
loss_sentiment, accuracy_sentiment = model_sentiment.evaluate(test_dataset_sentiment)
print(f"Sentiment Classification - Loss: {loss_sentiment}")
print(f"Sentiment Classification - Accuracy: {accuracy_sentiment}")

# Neural Machine Translation (NMT)
# Pra-pemrosesan data untuk NMT
tokenizer_nmt = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (text.numpy() for text, _ in train_dataset), target_vocab_size=2**13
)

def encode_nmt(indonesian_text, english_text):
    indonesian_tokens = [tokenizer_nmt.vocab_size] + tokenizer_nmt.encode(
        indonesian_text.numpy()) + [tokenizer_nmt.vocab_size+1]
    english_tokens = [tokenizer_nmt.vocab_size] + tokenizer_nmt.encode(
        english_text.numpy()) + [tokenizer_nmt.vocab_size+1]
    return indonesian_tokens, english_tokens

def filter_max_length_nmt(x, y, max_length=max_length):
    return tf.logical_and(tf.size(x) <= max_length, tf.size(y) <= max_length)

train_dataset_nmt = tf.data.Dataset.zip((train_dataset.map(lambda x, y: (y, x)).map(encode_nmt).filter(filter_max_length_nmt).padded_batch(64),
                                         train_dataset.map(encode_sentiment).filter(filter_max_length_sentiment).padded_batch(32).prefetch(tf.data.experimental.AUTOTUNE)))
test_dataset_nmt = tf.data.Dataset.zip((test_dataset.map(lambda x, y: (y, x)).map(encode_nmt).filter(filter_max_length_nmt).padded_batch(64),
                                        test_dataset.map(encode_sentiment).filter(filter_max_length_sentiment).padded_batch(32).prefetch(tf.data.experimental.AUTOTUNE)))

# Membangun model NMT
encoder_vocab_size = tokenizer_nmt.vocab_size + 2
decoder_vocab_size = tokenizer_nmt.vocab_size + 2

encoder_embedding_dim = 256
decoder_embedding_dim = 256
rnn_units = 1024

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

encoder = Encoder(encoder_vocab_size, encoder_embedding_dim, rnn_units, batch_size)

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

attention_layer = BahdanauAttention(10)

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

decoder = Decoder(decoder_vocab_size, decoder_embedding_dim, rnn_units, batch_size)

# Definisi optimizer dan loss function
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([tokenizer_nmt.vocab_size] * BATCH_SIZE, 1)

        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

# Melatih model NMT
num_epochs_nmt = 10
for epoch in range(num_epochs_nmt):
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(train_dataset_nmt):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')

    print(f'Epoch {epoch+1} Loss {total_loss/len(train_dataset_nmt):.4f}')

# Fungsi untuk menerjemahkan teks Indonesia ke Inggris
def translate(indonesian_text):
    indonesian_text = tf.convert_to_tensor([tokenizer_nmt.vocab_size] + tokenizer_nmt.encode(indonesian_text) + [tokenizer_nmt.vocab_size+1])
    result = ''

    hidden = [tf.zeros((1, rnn_units))]
    enc_out, enc_hidden = encoder(indonesian_text, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([tokenizer_nmt.vocab_size], 0)

    for t in range(max_length):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        if tokenizer_nmt.decode([predicted_id]) == '<end>':
            return result

        result += tokenizer_nmt.decode([predicted_id])

        dec_input = tf.expand_dims([predicted_id], 0)

    return result

# Contoh penerjemahan teks
indonesian_text = 'produk ini sangat bagus'
english_translation = translate(indonesian_text)
print(f'Indonesian Text: {indonesian_text}')
print(f'English Translation: {english_translation}')

# Klasifikasi sentimen untuk teks yang telah diterjemahkan
translated_text, _ = preprocess_sentiment(english_translation, 0)
encoded_translated_text, _ = encode_sentiment(translated_text, 0)
sentiment = model_sentiment.predict(tf.expand_dims(encoded_translated_text, 0))
print(f'Sentiment: {"Positive" if sentiment[0][0] > 0.5 else "Negative"}')
